# Estimación del área del espejo de agua

Este notebook implementa la estimación del área del espejo de agua de la Laguna de Fúquene utilizando imágenes Landsat y Google Earth Engine.

El foco está en la reproducibilidad del método y en la interpretación crítica de los resultados.

In [ ]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt

ee.Initialize()

## Área de interés

Se utiliza el polígono definido en el notebook anterior como Área de Interés (AOI).

In [ ]:
# Reemplazar por el asset real si aplica
roi = ee.FeatureCollection('users/tu_usuario/fuquene_roi').geometry()

## Colección Landsat

Se utiliza Landsat por su consistencia temporal y cobertura histórica.

In [ ]:
landsat = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filterBounds(roi) \
    .filterDate('1985-01-01', '2011-12-31')

## Preprocesamiento

Se aplican factores de escala y se calculan índices espectrales.

In [ ]:
def preprocess(img):
    green = img.select('SR_B2').multiply(0.0000275).add(-0.2)
    nir = img.select('SR_B4').multiply(0.0000275).add(-0.2)
    ndwi = green.subtract(nir).divide(green.add(nir)).rename('NDWI')
    return img.addBands(ndwi)

## Máscara de agua

Se clasifica agua utilizando un umbral fijo de NDWI.

In [ ]:
def water_mask(img):
    water = img.select('NDWI').gt(0.1)
    return water.rename('water')

## Cálculo del área anual

El área se calcula contando píxeles clasificados como agua y multiplicando por el área de cada píxel.

In [ ]:
def water_area_year(year):
    year = ee.Number(year)
    start = ee.Date.fromYMD(year, 1, 1)
    end = start.advance(1, 'year')

    col = landsat.filterDate(start, end).map(preprocess)

    img = ee.Image(col.median())

    water = water_mask(img)

    area_img = water.multiply(ee.Image.pixelArea())

    area = area_img.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=roi,
        scale=30,
        maxPixels=1e9
    ).get('water')

    return ee.Feature(None, {
        'year': year,
        'water_area_km2': ee.Algorithms.If(
            area,
            ee.Number(area).divide(1e6),
            None
        )
    })

## Serie temporal

In [ ]:
years = ee.List.sequence(1985, 2011)
fc = ee.FeatureCollection(years.map(water_area_year))

df = geemap.ee_to_df(fc)
df = df.sort_values('year')
df.head()

## Visualización

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(df['year'], df['water_area_km2'])
plt.xlabel('Año')
plt.ylabel('Área del espejo de agua (km²)')
plt.title('Evolución del área del espejo de agua - Laguna de Fúquene')
plt.grid(True)
plt.tight_layout()
plt.show()

## Nota final

Las estimaciones obtenidas representan una aproximación basada en datos EO y decisiones metodológicas explícitas.

El valor del análisis reside en la consistencia del método y en la interpretación crítica de sus resultados.